<a href="https://colab.research.google.com/github/Sandeep-4469/Chexpert_solution/blob/main/dsa_single_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def create_new_structure(output_dir, diseases):
    for disease in diseases:
        # Create new disease folder structure in the output directory
        disease_folder = os.path.join(output_dir, f"{disease}_")
        os.makedirs(os.path.join(disease_folder, 'train', f'{disease}'), exist_ok=True)
        os.makedirs(os.path.join(disease_folder, 'train', 'no_finding'), exist_ok=True)
        os.makedirs(os.path.join(disease_folder, 'valid', f'{disease}'), exist_ok=True)
        os.makedirs(os.path.join(disease_folder, 'valid', 'no_finding'), exist_ok=True)

        disease_train_path = os.path.join(train_dir, disease)
        if os.path.exists(disease_train_path):
            for file_name in os.listdir(disease_train_path):
                src = os.path.join(disease_train_path, file_name)
                dst = os.path.join(disease_folder, 'train', f'{disease}', file_name)
                shutil.copy(src, dst)

        disease_valid_path = os.path.join(valid_dir, disease)
        if os.path.exists(disease_valid_path):
            for file_name in os.listdir(disease_valid_path):
                src = os.path.join(disease_valid_path, file_name)
                dst = os.path.join(disease_folder, 'valid', f'{disease}', file_name)
                shutil.copy(src, dst)

        # Copy 'No_Finding' images into 'no_finding'
        no_finding_train_path = os.path.join(train_dir, 'No_Finding')
        if os.path.exists(no_finding_train_path):
            for file_name in os.listdir(no_finding_train_path):
                src = os.path.join(no_finding_train_path, file_name)
                dst = os.path.join(disease_folder, 'train', 'no_finding', file_name)
                shutil.copy(src, dst)

        no_finding_valid_path = os.path.join(valid_dir, 'No_Finding')
        if os.path.exists(no_finding_valid_path):
            for file_name in os.listdir(no_finding_valid_path):
                src = os.path.join(no_finding_valid_path, file_name)
                dst = os.path.join(disease_folder, 'valid', 'no_finding', file_name)
                shutil.copy(src, dst)

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

import shutil

train_dir = '/kaggle/input/chexpert/Dataset/train'
valid_dir = '/kaggle/input/chexpert/Dataset/valid'
output_dir = '/kaggle/working'

diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural_Effusion', 'Pneumonia']

create_new_structure(output_dir, diseases)

print("New folder structure created successfully.")


New folder structure created successfully.


In [ ]:
# Paths to directories and other configurations
output_dir = '/kaggle/working/chexpert_reorganized'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

batch_size = 32
epochs = 10
learning_rate = 1e-3

# Define data augmentation and preprocessing
train_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# List of diseases
diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural_Effusion', 'Pneumonia']
fine_tuned_models = []

In [ ]:
# Loop through each disease
for disease in diseases:
    disease_train_dir = f'/kaggle/working/{disease}_/train'
    disease_valid_dir = f'/kaggle/working/{disease}_/valid'

    if not os.path.exists(disease_train_dir) or not os.path.exists(disease_valid_dir):
        print(f"Data for {disease} not found in the expected directory structure.")
        continue

    # Create train and validation datasets and dataloaders
    train_dataset = ImageFolder(disease_train_dir, transform=train_transforms)
    valid_dataset = ImageFolder(disease_valid_dir, transform=valid_transforms)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # Model setup with DenseNet and additional layers
    base_model = models.densenet121(pretrained=True)
    for param in list(base_model.parameters())[:-5]:
        param.requires_grad = False  # Freeze all except the last 5 layers

    # Modify the last layers of DenseNet121 model
    num_features = base_model.classifier.in_features
    base_model.classifier = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )

    # Move model to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    base_model.to(device)

    # Define loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(base_model.parameters(), lr=learning_rate)

    best_val_accuracy = 0.0  # Track the best validation accuracy
    best_model = None

    # Training loop
    for epoch in range(epochs):
        base_model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        with tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}", ncols=100) as t:
            for inputs, labels in t:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = base_model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1).float())

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                preds = (outputs > 0.5).float()
                correct += (preds == labels.unsqueeze(1)).sum().item()
                total += labels.size(0)

                t.set_postfix(loss=running_loss / (t.n + 1), accuracy=100 * correct / total)

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct / total * 100
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation
        base_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with tqdm(valid_loader, desc=f"Validating Epoch {epoch + 1}/{epochs}", ncols=100) as t:
            with torch.no_grad():
                for inputs, labels in t:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = base_model(inputs)
                    loss = criterion(outputs, labels.unsqueeze(1).float())

                    val_loss += loss.item()
                    preds = (outputs > 0.5).float()
                    correct += (preds == labels.unsqueeze(1)).sum().item()
                    total += labels.size(0)

                    t.set_postfix(loss=val_loss / (t.n + 1), accuracy=100 * correct / total)

        val_loss /= len(valid_loader)
        val_accuracy = correct / total * 100
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

        # Update the best model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model = base_model.state_dict()

    # Save the best model for the current disease
    model_path = os.path.join(output_dir, f"{disease}_best_fine_tuned_model.pth")
    torch.save(best_model, model_path)
    print(f"Best fine-tuned model for {disease} saved successfully at {model_path}.")

    # Store the best model in the list
    fine_tuned_models.append(best_model)

print("All best models have been fine-tuned and saved.")


Training Epoch 1/10: 100%|███████████| 1250/1250 [01:34<00:00, 13.29it/s, accuracy=68.9, loss=0.585]


Epoch [1/10], Loss: 0.5854, Accuracy: 68.91%


Validating Epoch 1/10: 100%|███████████████| 4/4 [00:00<00:00,  6.67it/s, accuracy=85.6, loss=0.404]


Validation Loss: 0.4040, Validation Accuracy: 85.59%


Training Epoch 2/10: 100%|███████████| 1250/1250 [01:34<00:00, 13.24it/s, accuracy=71.1, loss=0.565]


...

Validation Loss: 0.3092, Validation Accuracy: 91.30%


Training Epoch 10/10: 100%|████████████| 814/814 [01:01<00:00, 13.17it/s, accuracy=80.3, loss=0.446]


Epoch [10/10], Loss: 0.4460, Accuracy: 80.34%


Validating Epoch 10/10: 100%|██████████████| 2/2 [00:00<00:00,  5.14it/s, accuracy=95.7, loss=0.178]

Validation Loss: 0.1782, Validation Accuracy: 95.65%
Best fine-tuned model for Pneumonia saved successfully at /kaggle/working/chexpert_reorganized/Pneumonia_best_fine_tuned_model.pth.
All best models have been fine-tuned and saved.


In [ ]:
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Path to saved models and validation images
output_dir = '/kaggle/working/chexpert_reorganized'  # Directory where models are saved
disease_valid_dirs = {
    'Atelectasis': '/kaggle/input/chexpert/Dataset/train/No_Finding',
    'Cardiomegaly': '/kaggle/input/chexpert/Dataset/valid/Cardiomegaly',
    'Consolidation': '/kaggle/input/chexpert/Dataset/valid/Consolidation',
    'Edema': '/kaggle/input/chexpert/Dataset/valid/Edema',
    'Pleural_Effusion': '/kaggle/input/chexpert/Dataset/valid/Pleural_Effusion',
    'Pneumonia': '/kaggle/input/chexpert/Dataset/valid/Pneumonia'
}

# Transform for validation images
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loop through each disease
for disease, valid_dir in disease_valid_dirs.items():
    # Update the model path to match the correct filename pattern
    model_path = os.path.join(output_dir, f"{disease}_best_fine_tuned_model.pth")

    # Initialize DenseNet121 model
    base_model = models.densenet121(pretrained=True)
    for param in list(base_model.parameters())[:-5]:
        param.requires_grad = False  # Freeze all except the last 5 layers

    # Modify the classifier layer
    num_features = base_model.classifier.in_features
    base_model.classifier = nn.Sequential(
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 1),
        nn.Sigmoid()
    )

    # Load model weights and move to device
    base_model.load_state_dict(torch.load(model_path, map_location=device))
    base_model.to(device)
    base_model.eval()

    # List files in the validation directory, filtering out directories
    image_files = [f for f in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, f))]
    if not image_files:
        print(f"No images found in {valid_dir}")
        continue

    # Loop to process only the first image
    for i, image_file in enumerate(image_files):
        if i >= 1:
            break  # Stop after processing the first image only

        image_path = os.path.join(valid_dir, image_file)
        image = Image.open(image_path).convert('RGB')
        image = valid_transforms(image).unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Make prediction
        with torch.no_grad():
            output = base_model(image)
            probability = output.item()
            prediction = "Positive" if probability > 0.5 else "Negative"

        print(f"{disease} - First image: {image_file} - Prediction: {prediction} - Probability: {probability:.4f}")


/tmp/ipykernel_31/2936342642.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_model.load_state_dict(torch.load(model_path, map_location=device))


Atelectasis - First image: patient14030_study5_view1_frontal.jpg - Prediction: Positive - Probability: 0.6232
Cardiomegaly - First image: patient64624_study1_view1_frontal.jpg - Prediction: Negative - Probability: 0.2684
Consolidation - First image: patient64624_study1_view1_frontal.jpg - Prediction: Negative - Probability: 0.3180
Edema - First image: patient64563_study1_view1_frontal.jpg - Prediction: Negative - Probability: 0.4270
Pleural_Effusion - First image: patient64644_study1_view1_frontal.jpg - Prediction: Negative - Probability: 0.0834
Pneumonia - First image: patient64692_study1_view1_frontal.jpg - Prediction: Negative - Probability: 0.3825


In [ ]:
/kaggle/working/chexpert_reorganized/Pneumonia_best_fine_tuned_model.pth

In [ ]:
import os

output_dir = '/kaggle/working/chexpert_reorganized'
print("Files in output_dir:", os.listdir(output_dir))

Files in output_dir: ['Consolidation_best_fine_tuned_model.pth', 'Pleural_Effusion_best_fine_tuned_model.pth', 'Atelectasis_best_fine_tuned_model.pth', 'Pneumonia_best_fine_tuned_model.pth', 'Cardiomegaly_best_fine_tuned_model.pth', 'Edema_best_fine_tuned_model.pth']


In [ ]:
train_dataset.classes

['Pneumonia', 'no_finding']

In [ ]:
import os
import zipfile

# Path to the directory where models are saved
output_dir = '/kaggle/working/chexpert_reorganized'

# Path to the zip file you want to create
zip_file_path = '/kaggle/working/fine_tuned_models.zip'

# Create a zip file containing all .pth files in the output directory
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for filename in os.listdir(output_dir):
        if filename.endswith('.pth'):
            model_file = os.path.join(output_dir, filename)
            zipf.write(model_file, os.path.basename(model_file))  # Add the model file to the zip

print(f"All .pth files have been zipped and saved to {zip_file_path}")


All .pth files have been zipped and saved to /kaggle/working/fine_tuned_models.zip
